In [1]:
# Import several packages that will be used throughout

# numeric packages
import numpy as np
import scipy
import scipy.io
import pandas as pd

# filesystem and OS
import sys, os, time
import glob

# images
import skimage
import skimage
import skimage.io

# plotting
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# these magics ensure that external modules that are modified are also automatically reloaded
%load_ext autoreload
%autoreload 2

In [2]:
import citymodel

In [2]:
from IPython import parallel

rc = parallel.Client()
all_engines = rc[:]
lbv = rc.load_balanced_view()

print len(all_engines)

48


# Generate samples

In [12]:
Hu_vec = np.linspace(40, 90, 5)
Hr_vec = 100 - Hu_vec
Gu =[2,2.2,2.4,2.6,2.8,3,3.2,3.4,3.6,3.8] # Urban gamma
Gr =[2,2.2,2.4,2.6,2.8,3,3.2,3.4,3.6,3.8] # Rural gamma
T_vec = np.linspace(0.01, 0.4, 10)        # threshold of built land
N_SIM = 1

params_vec = np.array(np.meshgrid(Hu_vec, Gu, Gr, T_vec, range(N_SIM)))\
                .T.reshape((-1,5))

params_vec.shape

(5000, 5)

In [13]:
%%px --local 
import os
import numpy as np
import skimage
import skimage.io

In [50]:
savepath = "/home/adalbert/data/world-cities/simulations/"

def generate_samples(params, savepath=savepath):
    Ue, Gu, Gr, T, n = params
    Ue = int(Ue)
    Ur = 100 - Ue
    nseed = 100
    max_niter = 100
    L = 300  # map size
    TRE=5 # threshold of area from rural to urban NOW IN NUMBER OF PIXEL
    iter_save = 3
    
    basedir = savepath + \
        "/Hu%d_Gu%2.2f_Gr%2.2f_T%2.2f"%(Ue, Gu, Gr, T)
    if not os.path.exists(basedir):
        os.makedirs(basedir)

    Mu0 = initialize_matrix(L, 1)
    Mr0 = initialize_matrix(L, nseed)
    M, M_dict = updated_ribsky_model(Mu0, Mr0, Gu, Gr, Ue, Ur, T, TRE, \
                             niter=max_niter)

    if iter_save is not None:
        ret = []
        for it, M in M_dict.iteritems():
            if it % iter_save == 0:
                filename = "%s/%d_iter%d.png"%(basedir,n,it)
                skimage.io.imsave(filename, M)
                ret.append(filename)
    else:
        filename = "%s/%d_iter%d.png"%(basedir,n,max(M_dict.keys()))
        skimage.io.imsave(filename, M)
        ret = [filename]
        
    return ret

In [48]:
generate_samples(params_vec[0], iter_save=3)

['/home/adalbert/data/world-cities/simulations//Hu40_Gu2.00_Gr2.00_T0.01/0_iter0.png',
 '/home/adalbert/data/world-cities/simulations//Hu40_Gu2.00_Gr2.00_T0.01/0_iter3.png',
 '/home/adalbert/data/world-cities/simulations//Hu40_Gu2.00_Gr2.00_T0.01/0_iter6.png']

In [55]:
amr = lbv.map_async(generate_samples, params_vec)

In [56]:
amr.progress

268

In [ ]:
amr.result()

In [18]:
params_vec[:10].shape

(10, 5)